In [1]:
from langchain.chat_models import  ChatOpenAI
from langchain.agents import initialize_agent,AgentType, Tool
from langchain.utilities import SQLDatabase
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.memory import ConversationBufferMemory
from typing import Type, List, Optional, Any
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# 获取 环境变量
#大模型apikey和url
api_key = os.getenv('api_key')
base_url = os.getenv('base_url')

#天气查询的apikey
juheAppKey = os.getenv('juheAppKey')

#数据库信息
mysql_info = os.getenv('mysql_info')

In [3]:
#定义大模型
online_model = ChatOpenAI(
    api_key=api_key,
    model="qwen-plus",
    base_url=base_url,
    temperature=0
)

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_20679/113483475.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  online_model = ChatOpenAI(


In [4]:
#定义数据库 连接
db = SQLDatabase.from_uri(mysql_info)


In [5]:
#查看数据库中的可用表
db.get_usable_table_names()

['city_stats']

In [6]:
#查看表结构
print(db.get_table_info())


CREATE TABLE city_stats (
	city_name VARCHAR(255), 
	population VARCHAR(255), 
	area VARCHAR(255)
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from city_stats table:
city_name	population	area
北京	20000000	400000234
上海	17689300	29867200
河北	19999944	39999821
*/


In [7]:
#查看数据库信息
print(db.get_context())

{'table_info': '\nCREATE TABLE city_stats (\n\tcity_name VARCHAR(255), \n\tpopulation VARCHAR(255), \n\tarea VARCHAR(255)\n)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4\n\n/*\n3 rows from city_stats table:\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821\n*/', 'table_names': 'city_stats'}


In [57]:


#定义数据库信息查询工具
class QueryDatabaseInfoInput(BaseModel):
    # query:str = Field(description="占位符")
    pass


class QueryDatabaseInfoTool(BaseTool):
    name:str = Field(default="get_database_info")
    description:str = Field(default="获取数据库信息")
    args_schema:Type[QueryDatabaseInfoInput] = QueryDatabaseInfoInput

    def _run(self) -> str:
        return db.get_context()
    
    def _arun(self) -> str:
        return "暂时不支持异步查询"
    


#定义数据库表查询工具
class QueryDatabaseTableInput(BaseModel):
    query:str = Field(description="sql查询语句")

class QueryDatabaseTableTool(BaseTool):
    name:str = Field(default="get_database_table")
    description:str = Field(default="根据 sql语句 获取数据库表信息")
    args_schema:Type[QueryDatabaseTableInput] = QueryDatabaseTableInput

    def _run(self, query:str) -> str:
        return db.run(query)

    def _arun(self, query:str) -> str:
        return "暂时不支持异步查询"
    


In [58]:
tools = [QueryDatabaseInfoTool(),QueryDatabaseTableTool()]

In [59]:
#初始化 记忆模块
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

In [63]:
agent = initialize_agent(
    tools,
    online_model,
    AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True, # 处理解析错误
    memory=memory
)

In [70]:
# 查看 记忆内容
agent.memory.chat_memory.messages

[HumanMessage(content='请帮我查询数据库信息', additional_kwargs={}, response_metadata={}),
 AIMessage(content='数据库信息如下：\n表名：city_stats\n表结构：\ncity_name (VARCHAR(255))\npopulation (VARCHAR(255))\narea (VARCHAR(255))\n示例数据：\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='请帮我查询一下表中的所有信息', additional_kwargs={}, response_metadata={}),
 AIMessage(content='请问您想查询哪个表的信息？请提供具体的表名以便我为您查询。', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='请告诉我数据库表city_stats中人口最多的两个城市名字是哪些，分别是多少人', additional_kwargs={}, response_metadata={}),
 AIMessage(content='人口最多的两个城市名字是北京和河北，人口数分别为20000000和19999944。', additional_kwargs={}, response_metadata={})]

In [65]:
result = agent.invoke("请帮我查询数据库信息")
result



> Entering new AgentExecutor chain...
Thought: 需要使用 get_database_info 工具来获取数据库信息

Action:
```
{
  "action": "get_database_info",
  "action_input": {}
}
``` 

Observation: {'table_info': '\nCREATE TABLE city_stats (\n\tcity_name VARCHAR(255), \n\tpopulation VARCHAR(255), \n\tarea VARCHAR(255)\n)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4\n\n/*\n3 rows from city_stats table:\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821\n*/', 'table_names': 'city_stats'}
Thought:我已经获取了数据库信息，现在可以回答用户的问题。

Action:
```
{
  "action": "Final Answer",
  "action_input": "数据库信息如下：\n表名：city_stats\n表结构：\ncity_name (VARCHAR(255))\npopulation (VARCHAR(255))\narea (VARCHAR(255))\n示例数据：\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821"
}
```

> Finished chain.


{'input': '请帮我查询数据库信息',
 'history': [HumanMessage(content='请帮我查询数据库信息', additional_kwargs={}, response_metadata={}),
  AIMessage(content='数据库信息如下：\n表名：city_stats\n表结构：\ncity_name (VARCHAR(255))\npopulation (VARCHAR(255))\narea (VARCHAR(255))\n示例数据：\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821', additional_kwargs={}, response_metadata={})],
 'output': '数据库信息如下：\n表名：city_stats\n表结构：\ncity_name (VARCHAR(255))\npopulation (VARCHAR(255))\narea (VARCHAR(255))\n示例数据：\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821'}

In [71]:
result = agent.invoke("请帮我查询一下表中的所有信息")
result



> Entering new AgentExecutor chain...
Thought: 用户想要查询表中的所有信息，但是没有提供具体的表名。我需要向用户询问表名。

Action:
```
{
  "action": "Final Answer",
  "action_input": "请问您想查询哪个表的信息？请提供表名以便我为您查询。"
}
```

> Finished chain.


{'input': '请帮我查询一下表中的所有信息',
 'history': [HumanMessage(content='请帮我查询数据库信息', additional_kwargs={}, response_metadata={}),
  AIMessage(content='数据库信息如下：\n表名：city_stats\n表结构：\ncity_name (VARCHAR(255))\npopulation (VARCHAR(255))\narea (VARCHAR(255))\n示例数据：\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='请帮我查询一下表中的所有信息', additional_kwargs={}, response_metadata={}),
  AIMessage(content='请问您想查询哪个表的信息？请提供具体的表名以便我为您查询。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='请告诉我数据库表city_stats中人口最多的两个城市名字是哪些，分别是多少人', additional_kwargs={}, response_metadata={}),
  AIMessage(content='人口最多的两个城市名字是北京和河北，人口数分别为20000000和19999944。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='请帮我查询一下表中的所有信息', additional_kwargs={}, response_metadata={}),
  AIMessage(content='请问您想查询哪个表的信息？请提供表名以便我为您查询。', additional_kwargs={}, response_metadata={})],
 'output'

In [72]:
result = agent.invoke("请告诉我数据库表city_stats中人口最多的两个城市名字是哪些，分别是多少人")
result



> Entering new AgentExecutor chain...
Thought: 首先，我需要从数据库中获取表city_stats的信息。为了做到这一点，我将使用get_database_table工具执行一条SQL查询语句，这条语句将选择人口数量降序排列的前两个城市的名字和人口数量。
Action:
```
{
  "action": "get_database_table",
  "action_input": {
    "query": "SELECT city_name, population FROM city_stats ORDER BY population DESC LIMIT 2;"
  }
}
```

Observation: [('北京', '20000000'), ('河北', '19999944')]
Thought:我已经从数据库中获取了人口最多的两个城市的名字和人口数量。现在我可以给出最终的答案。
Action:
```
{
  "action": "Final Answer",
  "action_input": "人口最多的两个城市是北京和河北，分别是20000000人和19999944人。"
}
```

> Finished chain.


{'input': '请告诉我数据库表city_stats中人口最多的两个城市名字是哪些，分别是多少人',
 'history': [HumanMessage(content='请帮我查询数据库信息', additional_kwargs={}, response_metadata={}),
  AIMessage(content='数据库信息如下：\n表名：city_stats\n表结构：\ncity_name (VARCHAR(255))\npopulation (VARCHAR(255))\narea (VARCHAR(255))\n示例数据：\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='请帮我查询一下表中的所有信息', additional_kwargs={}, response_metadata={}),
  AIMessage(content='请问您想查询哪个表的信息？请提供具体的表名以便我为您查询。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='请告诉我数据库表city_stats中人口最多的两个城市名字是哪些，分别是多少人', additional_kwargs={}, response_metadata={}),
  AIMessage(content='人口最多的两个城市名字是北京和河北，人口数分别为20000000和19999944。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='请帮我查询一下表中的所有信息', additional_kwargs={}, response_metadata={}),
  AIMessage(content='请问您想查询哪个表的信息？请提供表名以便我为您查询。', additional_kwargs={}, respons